# Online Retail Sales Prediction Modeling

## Timeseries Models

## Regression Models

## Classification Models



### Setup Libraries and Load in Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
country_mapping_df = pd.read_csv("data/Country Mapping.csv")
region_mapping_df = pd.read_csv("data/region_mapping.csv")
retail_sales_df = pd.read_csv("data/Online Retail.csv")
# Add New Features
retail_sales_df["Cost"] = retail_sales_df["UnitPrice"]*retail_sales_df["Quantity"]
retail_sales_df['alpha-3'] = pd.merge(retail_sales_df,country_mapping_df,how='left',left_on='Country',right_on='country_name').drop(['country_name'],axis=1)['alpha-3']
retail_sales_df = pd.merge(retail_sales_df,region_mapping_df,how='left',on='alpha-3')

In [3]:
retail_sales_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Cost,alpha-3,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30,GBR,United Kingdom,GB,826.0,ISO 3166-2:GB,Europe,Northern Europe,NaN,150.0,154.0,NaN
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,GBR,United Kingdom,GB,826.0,ISO 3166-2:GB,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00,GBR,United Kingdom,GB,826.0,ISO 3166-2:GB,Europe,Northern Europe,NaN,150.0,154.0,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,GBR,United Kingdom,GB,826.0,ISO 3166-2:GB,Europe,Northern Europe,NaN,150.0,154.0,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,GBR,United Kingdom,GB,826.0,ISO 3166-2:GB,Europe,Northern Europe,NaN,150.0,154.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20,FRA,France,FR,250.0,ISO 3166-2:FR,Europe,Western Europe,NaN,150.0,155.0,NaN
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60,FRA,France,FR,250.0,ISO 3166-2:FR,Europe,Western Europe,NaN,150.0,155.0,NaN
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60,FRA,France,FR,250.0,ISO 3166-2:FR,Europe,Western Europe,NaN,150.0,155.0,NaN
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60,FRA,France,FR,250.0,ISO 3166-2:FR,Europe,Western Europe,NaN,150.0,155.0,NaN


In [9]:
#TODO Clean up cancelled orders 
total_invoice_predictions = retail_sales_df[["InvoiceNo","InvoiceDate"]].drop_duplicates()
total_invoice_predictions["InvoiceDate"] = pd.to_datetime(total_invoice_predictions["InvoiceDate"])
total_invoice_predictions

,InvoiceNo,InvoiceDate
0,536365,2010-12-01 08:26:00
7,536366,2010-12-01 08:28:00
9,536367,2010-12-01 08:34:00
21,536368,2010-12-01 08:34:00
25,536369,2010-12-01 08:35:00
...,...,...
541865,581583,2011-12-09 12:23:00
541867,581584,2011-12-09 12:25:00
541869,581585,2011-12-09 12:31:00
541890,581586,2011-12-09 12:49:00


In [13]:
total_invoice_predictions_by_day = total_invoice_predictions.groupby(pd.Grouper(key='InvoiceDate', axis=0,  freq='D')).count().reset_index().rename(columns={"InvoiceNo":"Count"})
total_invoice_predictions_by_day

,InvoiceDate,Count
0,2010-12-01,144
1,2010-12-02,167
2,2010-12-03,108
3,2010-12-04,0
4,2010-12-05,95
...,...,...
369,2011-12-05,151
370,2011-12-06,149
371,2011-12-07,138
372,2011-12-08,145


In [ ]:
# Set aside the last 36 days as a validation series
train, val = total_invoice_predictions_by_day[:-36], total_invoice_predictions_by_day[-36:]

In [ ]:
from darts.models import ExponentialSmoothing

model = ExponentialSmoothing()
model.fit(train)
prediction = model.predict(len(val), num_samples=1000)

In [ ]:
series.plot()
prediction.plot(label="forecast", low_quantile=0.05, high_quantile=0.95)
plt.legend()